In [1]:
import os
import tensorflow as tf
import numpy as np

In [2]:

keymap={'1':0,'2':1,'3':2,'4':3,'5':4,'6':5,'7':6,'8':7,'9':8,
        'A':10,'B':11,'C':12,'D':13,'E':14,'F':15,'G':16,'H':17,
        'I':18,'J':19,'K':20,'L':21,'M':22,'N':23,'O':24,'P':25,
        'Q':26,'R':27,'S':28,'T':29,'U':30,'V':31,'W':32,'X':33,'Y':34,'Z':35}
input_dir='data'
output_dir='processed'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
model_path = 'hand_landmarker.task'

In [3]:
import mediapipe as mp
from tensorflow.keras.preprocessing.image import load_img,img_to_array

def extrLandmarks(image_path):
    hands = mp.solutions.hands
    hands_mesh = hands.Hands(static_image_mode=True, min_detection_confidence=0.7) #hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
    img = load_img(image_path)
    img_array = img_to_array(img, dtype='uint8') # a numpy array
    results = hands_mesh.process(img_array)
    lm=[]
    for res in results.multi_hand_landmarks:
        for data_point in res.landmark:
            lm.extend([data_point.x, data_point.y, data_point.z])
    if len(lm) == 63:
        lm.extend([0]*63)
    lm=np.array(lm)
    return lm

In [3]:
import os
import numpy as np
import mediapipe as mp
from tensorflow.keras.utils import Sequence
import cv2

class DataGenerator(Sequence):
    def __init__(self, input_dir, output_dir, batch_size=32):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.batch_size = batch_size
        self.file_list = self._generate_file_list()
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=True, max_num_hands=2)
        self.mp_drawing = mp.solutions.drawing_utils
    
    def _generate_file_list(self):
        file_list = []
        for root, _, files in os.walk(self.input_dir):
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg')):
                    file_list.append(os.path.join(root, file))
        return file_list

    def __len__(self):
        return int(np.ceil(len(self.file_list) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_files = self.file_list[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_landmarks = []
        
        for file in batch_files:
            image = cv2.imread(file)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = self.hands.process(image_rgb)
            
            landmarks = []
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks.extend([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
            
            if landmarks:
                batch_landmarks.append(np.array(landmarks))
                output_path = self._get_output_path(file)
                self._save_landmarks(output_path, np.array(landmarks))
                print(f'Landmarks saved to {output_path}')
        
        return batch_landmarks

    def _get_output_path(self, file_path):
        relative_path = os.path.relpath(file_path, self.input_dir)
        output_path = os.path.join(self.output_dir, os.path.splitext(relative_path)[0] + '.npy')
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        return output_path

    def _save_landmarks(self, output_path, landmarks):
        np.save(output_path, landmarks)

    def on_epoch_end(self):
        np.random.shuffle(self.file_list)


data_gen = DataGenerator(input_dir, output_dir, batch_size=50)
for batch in data_gen:
    # 
    pass

c:\Media files\Coding\GIT\SLD\sldenv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Landmarks saved to processed\1\0.npy
Landmarks saved to processed\1\1.npy
Landmarks saved to processed\1\10.npy
Landmarks saved to processed\1\100.npy
Landmarks saved to processed\1\1000.npy
Landmarks saved to processed\1\1001.npy
Landmarks saved to processed\1\1002.npy
Landmarks saved to processed\1\1003.npy
Landmarks saved to processed\1\1004.npy
Landmarks saved to processed\1\1005.npy
Landmarks saved to processed\1\1006.npy
Landmarks saved to processed\1\1007.npy
Landmarks saved to processed\1\1008.npy
Landmarks saved to processed\1\1009.npy
Landmarks saved to processed\1\101.npy
Landmarks saved to processed\1\1010.npy
Landmarks saved to processed\1\1011.npy
Landmarks saved to processed\1\1012.npy
Landmarks saved to processed\1\1013.npy
Landmarks saved to processed\1\1014.npy
Landmarks saved to processed\1\1015.npy
Landmarks saved to processed\1\1016.npy
Landmarks saved to processed\1\1017.npy
Landmarks saved to processed\1\1018.npy
Landmarks saved to processed\1\1019.npy
Landmarks 